In [25]:
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urlunparse
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import os

links=[]
with open(r"Links\links_peru21.txt","r",encoding="utf-8") as file:
    for line in file:
        links.append(line.replace('\n',''))


In [26]:
print(len(links))
links=list(set(links))
print(len(links))


87328
71414


In [2]:
link=links[0]
link

'https://peru21.pe/lima/controlo-incendio-almacen-material-reciclado-puente-piedra-64457-noticia/'

In [14]:
def remove_scripts(soup):
    for script in soup(["script", "style"]):
        script.extract()

def fetch_content(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')
        remove_scripts(soup)
        headline_tag = soup.find('h1', itemprop='name')
        headline = headline_tag.text.strip() if headline_tag else ''
        print(headline)
        article_body = soup.find('div', class_='story-contents__content').text
        print(article_body)
        date_tag = soup.find('time')
        date = date_tag['datetime'] if date_tag else ''
        try:
            date = datetime.fromisoformat(date).strftime('%d-%m-%Y')
        except ValueError:
            try:
                date = datetime.strptime(date, '%a %b %d %Y %H:%M:%S GMT%z (%Z)').strftime('%d-%m-%Y')
            except ValueError:
                pass
        print(date)

        tags_tag = soup.find('div', class_='st-tags__box')
        tags = set(tag.text for tag in tags_tag.find_all('a')) if tags_tag else set()
        tags_str = ', '.join(tags)
        print(tags_str)

        bajada_tag = soup.find('h2', class_='sht__summary')
        bajada = bajada_tag.text.strip() if bajada_tag else ''
        print(bajada)
        antetitulo_tag = soup.find('div', class_='st-social f just-between')
        antetitulo = antetitulo_tag.text.strip() if antetitulo_tag else ''
        print(antetitulo)
        print(link, headline.replace('\n', ''), bajada.replace('\n', ''), antetitulo.replace('\n', ''), article_body.replace('\n', ''), date, tags_str)
    except Exception as e:
        print(f"Error processing {link}: {e}")



In [13]:
with ThreadPoolExecutor() as executor, open("peru21.csv", 'w', encoding='UTF-8') as doc_csv:
    doc_csv.write("link^^headline^^bajada^^antetitulo^^cuerpo^^dia^^etiquetas\n")
    for result in executor.map(fetch_content, links):
        if result:
            doc_csv.write(f"{'^^'.join(result)}\n")

Se controló incendio en almacén de material reciclado en Puente Piedra
Cinco unidades de los bomberos se necesitaron para apagar el incendio en un local de material reciclado  de nombre Jv plásticos en Puente Piedra a la altura del kilómetro 27 de la Panamericana Norte.Los bomberos llegaron rápidamente al local, ya que se dirigían al incendio que se desarrolla en Carabayllo en un depósito de aserrín, por lo que pudieron controlar la emergencia rápidamente. Se usaron cinco unidades.  "Las unidades se estaban retirándose para el relevo, porque la emergencia en Carabayllo sigue, y (esas) son las que están acá, por lo que no hemos necesitado trabajar con hidrantes porque todas las unidades han estado recargadas, y por la cantidad de material nos ha sido suficientes (con esa agua)", indicó el bombero Enrique Egusquiza a RPP.Enrique Egusquiza manifestó que casi todo el material del local se ha visto comprometido, pero que ya se controló la emergencia.DATO Casi en paralelo se desarrolló un in

KeyboardInterrupt: 

In [ ]:
def extract_data(link):
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        headline_tag = soup.find('h1', itemprop='name')
        headline = headline_tag.text.strip() if headline_tag else ''

        paragraphs = soup.find_all('p', itemprop='description')
        article_body = ' '.join(paragraph.text.strip() for paragraph in paragraphs)

        date_tag = soup.find('time')
        date = date_tag['datetime'] if date_tag else ''
        try:
            date = datetime.fromisoformat(date).strftime('%d-%m-%Y')
        except ValueError:
            try:
                date = datetime.strptime(date, '%a %b %d %Y %H:%M:%S GMT%z (%Z)').strftime('%d-%m-%Y')
            except ValueError:
                pass

        tags_tag = soup.find('div', class_='st-tags__box')
        tags = set(tag.text for tag in tags_tag.find_all('a')) if tags_tag else set()
        tags_str = ', '.join(tags)

        bajada_tag = soup.find('h2', class_='sht__summary')
        bajada = bajada_tag.text.strip() if bajada_tag else ''

        antetitulo_tag = soup.find('div', class_='st-social f just-between')
        antetitulo = antetitulo_tag.text.strip() if antetitulo_tag else ''

        return (link, headline.replace('\n', ''), bajada.replace('\n', ''), antetitulo.replace('\n', ''), article_body.replace('\n', ''), date, tags_str)
    except Exception as e:
        print(f"Error processing {link}: {e}")
        return None

with ThreadPoolExecutor() as executor, open("peru21.csv", 'a', encoding='UTF-8') as doc_csv:
    for result in executor.map(extract_data, links):
        if result:
            doc_csv.write(f"{';'.join(result)}\n")